*** If unfamiliar with Jupyter Notebooks, instructions for installing and running can be found here: http://jupyter.org/install. Before installing Jupyter Notebook, make sure that Python is installed (our code is with Python3) in your system. We recommend installing Python and Jupyter using the conda package manager.***


# Data Prepration

We need to take a few steps to convert the raw data (forum text data) to the one (LIWC feature vectos) that we can use for our salient social identity detection model:

1- Cleaning

2- Importing to database

3- Output processed text for input to LIWC

4- Processing for LIWC features

5- Importing LIWC vectors back into db

6- Sampling LIWC vectors to csv file

## Cleaning 

These are the steps I used to preprocesses raw csv files to remove copy artifacts, and make good for import into the database. These steps are particularly relevant to the Mumsnet dataset and Reddit dataset, but they may be helpful for other data too.

### Removing new lines
by removing the new lines we convert each message into a single line, which is compatible with LIWC software. First I made a copy of the csv files, for example ParentMessages.csv (copies have $\_$ clean added to file stem), then I run the following sed commands to clean them up: 

### Dropping deleted posts

Sometimes a post is deleted by the moderator of forum or by user herself. 


In case of mumsnet, these posts could be detected by matching with patterns like "Message deleted", 
"Message deleted by Mumsnet", "Message withdrawn", "Message withdrawn at poster's request". 


For reddit data, there are patterns like "[deleted]", "[removed]", "[deleted by user]". 
We also drop the posts which belong to users who are no longer in the platform, by matching with patterns like
"[deleted]", and "[removed]". 


### Dropping bots posts
There are some posts generated with bots and contains patterns like  "I'm a bot...", . This is particularly relevant to reddit data but it may be helpful for other data too.


We run regular expression to detect and drop these posts.


In [38]:
# path to the csv files
csv_path = './preprocessing_test/raw_data/'
file_name = 'comments_test'

In [39]:
import pandas as pd

# reading reddit file
test = pd.read_csv(csv_path+file_name+'.csv', encoding='utf-8')

test = test[['author', 'id', 'body']]


In [40]:
#regelar expressions to find the matched patterns
mumsnet_regex_1 = "message deleted"
mumsnet_regex_2 = "message deleted by mumsnet."
mumsnet_regex_3 = "message deleted by mumsnet for breaking our talk guidelines."
mumsnet_regex_4 = "message withdrawn at poster's request."
mumsnet_regex_5 = "message withdrawn by mumsnet."


reddit_regex_1 = '\[deleted\]'
reddit_regex_2 = '\[removed\]'
reddit_regex_3 = '(?i)I\'m a bot'
reddit_regex_4 = '(?i)I ?\^?(\'?m|am) \^?\^?a \^?\^?bot'
reddit_regex_5 = '(?i)this bot wants to find'
reddit_regex_6 = '\[deleted by user\]'

In [41]:
import re

def regex_match(text, regex):
    if not (re.search(regex, text) is None):
        return True
    else:
        return False
    
    
def reddit_cm_cleaning(text):
    proctext = str(text).lower()
    if (regex_match(proctext, reddit_regex_1) or regex_match(proctext, reddit_regex_2) 
        or regex_match(proctext, reddit_regex_3) or regex_match(proctext, reddit_regex_4)
        or regex_match(proctext, reddit_regex_5) or regex_match(proctext, reddit_regex_6)):
        return True
    else:
        return False
    
    
def reddit_author_cleaning(text):
    proctext = str(text).lower()
    if (regex_match(proctext, reddit_regex_1) or regex_match(proctext, reddit_regex_2)):
        return True
    else:
        return False
    
    
def mumsnet_cm_cleaning(text):
    proctext = str(text).lower().strip()
    if (regex_match(proctext, mumsnet_regex_1) or regex_match(proctext, mumsnet_regex_2) 
        or regex_match(proctext, mumsnet_regex_3) or regex_match(proctext, mumsnet_regex_4)
        or regex_match(proctext, mumsnet_regex_5)):
        return True
    else:
        return False


In [42]:
def reddit_cleaned_mgs(df):
    df['cleaned'] = df['body'].apply(reddit_cm_cleaning)
    idlist = df.loc[df['cleaned'] == False]['id']
    
    f_df = df.loc[df['id'].isin(idlist)]
    
    f_df['cleaned'] = f_df['author'].apply(reddit_author_cleaning)
    idlist = f_df.loc[f_df['cleaned'] == False]['id']
    
    f_df = f_df.loc[df['id'].isin(idlist)]
    
    return f_df[['author', 'id', 'body']]
   

In [43]:
def mumsnet_cleaned_mgs(df):
    df['cleaned'] = df['body'].apply(mumsnet_cm_cleaning)
    idlist = df.loc[df['cleaned'] == False]['id']
    
    f_df = df.loc[df['id'].isin(idlist)]
    
    return f_df[['author', 'id', 'body']]



In [44]:
df = reddit_cleaned_mgs(test)
df = mumsnet_cleaned_mgs(df)

df.to_csv(csv_path + file_name + '_cleaned.csv', index=False)







There are a few other cleaning steps which are done in the export from db to LIWC input functionality.

## Importing to the database

Next step is importing the cleaned csv files into the database. During this step, we create a unique user_id for each user_name in our database.

Having our data in format of a table gives us the following advantagous:

- visualization (having access to different parts our data)
- managable (updating our dataset)
- searchable (searching for a specific user or post by mysql commands)

for this, we first need to install sqlite3. SQLite is a relational database management system that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine. In contrast to many other database management systems, SQLite is not a client–server database engine. Rather, it is embedded into the end program.  SQLite is the most used database engine in the world, and it is built into all mobile phones and most computers and comes bundled inside countless other applications that people use every day.  instructions for installing and running can be found here: https://mislav.net/rails/install-sqlite3/  



In [45]:
import os
import sqlite3


def get_path(fpath=None, fdir=None, fstem=None, fext=None):
    
    if fpath is None:
        fpath = fdir + os.sep + fstem + '.' + fext
    fpath = os.sep.join(fpath.split('/'))
    
    return fpath


def connect_to_db(dbdir, dbstem, dbpath=None):
    print("Connecting to db with dbdir = %s" % dbdir)
    print("Connecting to db with dbstem = %s" % dbstem)
    
    dbpath = get_path(
        fpath=dbpath, fdir=dbdir, fstem=dbstem, fext='db')
    conn = sqlite3.connect(dbpath)
    
    return conn


def create_tables(conn, cursor):

    query = """
        CREATE TABLE IF NOT EXISTS comments(
            comment_id varchar(255) PRIMARY KEY,
            author_id varchar(255), body TEXT)
        """

    cursor.execute(query)

    query = """
        CREATE TABLE IF NOT EXISTS authors(
            author_id INTEGER PRIMARY KEY, author_name varchar(255),
            FOREIGN KEY(author_id) REFERENCES comments(author_id))
        """

    cursor.execute(query)

    conn.commit()
    return conn, cursor



def create_db(dbdir, dbstem):
    conn = connect_to_db(dbdir=dbdir, dbstem=dbstem)
    cursor = conn.cursor()
    create_tables(conn, cursor)
    


In [46]:
DBDIR = './preprocessing_test/sqlit_database'
DBSTEM = 'comment_test'
SUBDIR = 'LIWC'

create_db(DBDIR, DBSTEM)

Connecting to db with dbdir = ./preprocessing_test/sqlit_database
Connecting to db with dbstem = comment_test


In [47]:
import csv

def write_comments_to_db(conn, cursor, file_name):
    with open(file_name, 'r') as handler:
        reader = csv.reader(handler, delimiter=',')
        # we ignore the first row
        firstrow = next(reader)
        print(firstrow)
        
        cursor.execute('BEGIN TRANSACTION')
        author_ids = {}
        count = 1
        for i, row in enumerate(reader):
            if len(row) == 0:
                continue
                
            try:
                author_name = str(row[0])                  
            except IndexError:
                raise
                
            try:
                comment_id = str(row[1])
            except IndexError:
                raise   
                
            try:
                body = row[2]
            except IndexError:
                raise
                

            if author_name in author_ids:
                author_id = author_ids[author_name]
            else:
                author_id = count
                author_ids[author_name] = author_id
                count += 1


            query_1 = """
                INSERT INTO comments(author_id, comment_id, body) VALUES(?,?,?)
                """

            query_2 = """
                INSERT INTO authors(author_name, author_id) VALUES(?,?)
                """

            try:
                cursor.execute(query_1, (author_id, comment_id, body))
            except sqlite3.IntegrityError:
                continue

            try:
                cursor.execute(query_2, (author_name, author_id))
            except sqlite3.IntegrityError:
                pass

        cursor.execute('COMMIT')



#writing comments into db
def prepare_comment_table(dbdir, dbstem, file_name):
    conn = connect_to_db(dbdir=dbdir, dbstem=dbstem)
    cursor = conn.cursor()
    write_comments_to_db(conn, cursor, file_name)   
    


In [48]:
prepare_comment_table(DBDIR, DBSTEM, csv_path+file_name+'_cleaned.csv')

Connecting to db with dbdir = ./preprocessing_test/sqlit_database
Connecting to db with dbstem = comment_test
['author', 'id', 'body']


In [49]:
import re
import sys


def make_directory(outdir):
    try:
        os.makedirs(outdir)
    except OSError as exc:  
        if exc.errno == os.errno.EEXIST and os.path.isdir(outdir):
            pass
        else:
            raise
            
            

def cursor_execute_Results(cursor, selectquery):
    cursor.execute(selectquery)

    user_results = [
        (str(post_id), str(body))
        for post_id, body in cursor.fetchall()]
    post_ids, texts = zip(*user_results)

    return post_ids, texts



def cleaning_for_LIWC(text, lowercase=True):
    WHITESPACEREGEX = r'[ \t\n\r\f\v]+'
    NONPUNCREGEX = r'[a-zA-Z0-9_\s]'
    
    if lowercase:
        proctext = text.lower()
    else:
        proctext = text
            
    proctext = proctext.strip()
    
    # now replace newlines with space 
    proctext.replace('\n',' ')

    
    # remove all duplicate whitespace
    proctext = re.sub(WHITESPACEREGEX, ' ', proctext)
    
    # if text is only punctuation, remove it
    if re.search(NONPUNCREGEX, proctext) is None:
        proctext = ''
        
    return proctext



def write_single_file(txtfpath, idfpath, post_ids, texts, 
                      lowercase=None, encode_as=None, parasep=None,
                      with_dummies=False, dummytext='xxxdummyxxx'):

    with open(txtfpath, 'w') as of_handler, open(idfpath, 'w') as idf_handler:
        # iterate through and write each to file
        # cursor.execute(selectquery)
        count = 0
        for i, (post_id, text) in enumerate(zip(post_ids, texts)):

            # if the text does not contain any words then skip it
            if re.search('[a-zA-Z]', text) is None:
                continue

            text = cleaning_for_LIWC(text,
                                     lowercase=lowercase)

            if len(text) == 0:
                continue

            # insert paragraph separator if line is greater than 0
            if count > 0:
                of_handler.write(parasep)
                
            # now encode text (if needed) and write to file
            if not encode_as is None:
                # text = text.encode(encode_as) #it's not working with python3
                text = str(text)
                
            # adding a dummy word at the enf of each post
            of_handler.write(text + ' ' + dummytext)  # without it, the liwc results was inconsistant with our data
            
            # if insertion of dummy lines required, do that here. it is optional
            if with_dummies:
                of_handler.write(parasep)
                of_handler.write(dummytext)

            idf_handler.write(str(post_id) + '\n')
            count += 1
            if (i % 10000) == 0:
                # print('post_id = %r' % (post_id,))
                sys.stdout.flush()
                
                
def query_clean_and_write_singlefiles(outdir, stem, cursor, selectquery, segmentation,
                                      lowercase, encode_as, parasep,
                                      with_dummies):
    if segmentation:
        post_ids, texts = cursor_execute_Results(cursor, selectquery)

        
        segment_size = 300000
        no_segments = int(len(post_ids) / segment_size)
        index = 0
        seg_no = 0
        while seg_no < no_segments:
            txtfpath = outdir + os.sep + stem + '_seg{}'.format(seg_no) + '.txt'
            idfpath = outdir + os.sep + stem + '_seg{}'.format(seg_no) + '.ids'

            post_ids_seg, texts_seg = post_ids[index:(seg_no + 1) * segment_size], texts[
                                                                                   index:(seg_no + 1) * segment_size]
            print(index, (seg_no + 1) * segment_size, "Writing to %s..." % (txtfpath,))

            write_single_file(txtfpath, idfpath, post_ids_seg, texts_seg, 
                              lowercase=lowercase, 
                              encode_as=encode_as, parasep=parasep, with_dummies=with_dummies)

            index = int((seg_no + 1) * segment_size)
            seg_no += 1

            if not test_numlines_txt_vs_ids(txtfpath, idfpath, with_dummies=with_dummies):
                raise ValueError('File lines do not match')

        if index < len(post_ids):
            txtfpath = outdir + os.sep + stem + '_seg{}'.format(seg_no) + '.txt'
            idfpath = outdir + os.sep + stem + '_seg{}'.format(seg_no) + '.ids'

            post_ids_seg, texts_seg = post_ids[index:], texts[index:]
            print(index, (seg_no + 1) * segment_size, "Writing to %s..." % (txtfpath,))
            write_single_file(txtfpath, idfpath, post_ids_seg, texts_seg, 
                              lowercase=lowercase, 
                              encode_as=encode_as, parasep=parasep, with_dummies=with_dummies)

            if not test_numlines_txt_vs_ids(txtfpath, idfpath, with_dummies=with_dummies):
                raise ValueError('File lines do not match')

    else:
        txtfpath = outdir + os.sep + stem + '.txt'
        idfpath = outdir + os.sep + stem + '.ids'
        print("Writing to %s..." % (txtfpath,))

        
        post_ids, texts = cursor_execute_Results(cursor, selectquery)


        write_single_file(txtfpath, idfpath, post_ids, texts, 
                          lowercase=lowercase, 
                          encode_as=encode_as, parasep=parasep, with_dummies=with_dummies)

        if not test_numlines_txt_vs_ids(txtfpath, idfpath, with_dummies=with_dummies):
            raise ValueError('File lines do not match')


            

def write_to_file(dbdir, dbstem, subdir, encode_as = None, segmentation=False, with_dummies=False):

    lowercase = True
    if encode_as is None:
        encode_as = 'utf-8'
        
    stem = 'minimal'
    parasep = '\n\n\n'

    outdir = dbdir + os.sep + subdir
    if not os.path.isdir(outdir):
        make_directory(outdir)
        
    conn = connect_to_db(dbdir=dbdir, dbstem=dbstem)
    cursor = conn.cursor()

    query = """
            SELECT comments.comment_id, comments.body
            FROM comments
            """

    query_clean_and_write_singlefiles(
        outdir, stem, cursor, query,
        lowercase=lowercase, segmentation=segmentation,
        encode_as=encode_as, parasep=parasep,
        with_dummies=with_dummies)
    

def test_numlines_txt_vs_ids(txtfpath, idfpath, with_dummies=False):
    numlines_txt = sum(1 for line in open(txtfpath))
    numlines_ids = sum(1 for line in open(idfpath))
    if not with_dummies:
        lengths_match = ((numlines_ids * 3) - 2) == numlines_txt
    else:
        lengths_match = ((numlines_ids * 6) - 2) == numlines_txt

    if lengths_match:
        print('File lengths are consistent, with %d items' % (numlines_ids,))
        return True
    else:
        print('File lengths are inconsistent')
        print('numlines: txt=%d, ids=%d' % (numlines_txt, numlines_ids))
        return False
    
#writing comments and posts from database to text file, to feed that to the LIWC
write_to_file(dbdir=DBDIR, dbstem=DBSTEM, subdir=SUBDIR, segmentation=True)


Connecting to db with dbdir = ./preprocessing_test/sqlit_database
Connecting to db with dbstem = comment_test
0 300000 Writing to ./preprocessing_test/sqlit_database/LIWC/minimal_seg0.txt...
File lengths are consistent, with 7 items


## Processing for LIWC features

This requires a copy of the LIWC software, and a copy of LIWC dictionary (we used LIWC_2007 dictionary). Next step is processing single file(s) with LIWC software. Select the categories you want the LIWC software output. Save the results in the same folder as the text and ids files (for example with label LIWC2007.csv).

 ## Importing LIWC vectors back into db
 To import LIWC's output csv files back into db, you can run the following code:

## Output processed text for input to LIWC

The next step is converting posts to the LIWC vectors. For this, we construct large files which can be consumed in a single processing run in the LIWC software. This was required as large numbers of input files (each with a single post) cause the software to choke. However, there were a number of issues with this approach too.
First, there are posts that contain no elements that are recognised by the LIWC dictionary (which excludes punctuation, so pure puctuation texts were ignored). For this, we can insert dummy lines between each posts (it is optional), which have a characteristic LIWC vector, so we can step through and ignore these when reading in the output from the LIWC. We also added a dummy word at the end of each post, to make sure of the consistency of the LIWC results and the input files.
Second, there is a limitation on the maximum size of the file which can be processed by LIWC software. Therefore, in case of large files, we split a single file into multiple smaller files. 

In [50]:
import csv

def get_path_(fpath=None, fdir=None, fstem=None, fext=None):
    if fpath is None:
        fpath = fdir + os.sep + fstem + '.' + fext
    fpath = os.sep.join(fpath.split('/'))
    return fpath


def create_liwc_table(conn, cursor, liwc_fields, liwctablename=None):
    parent_table = 'comments'
    template = ', '.join(['%s FLOAT'] * len(liwc_fields))
    liwc_columns_def = template % tuple(liwc_fields)
    query = """
        CREATE TABLE IF NOT EXISTS %s(
            comment_id varchar(255) PRIMARY KEY, %s,
            FOREIGN KEY(comment_id) REFERENCES %s(comment_id))
        """ % (liwctablename, liwc_columns_def, parent_table,)
    cursor.execute(query)
    conn.commit()


def get_liwc_table_insert_query(liwc_fields, liwctablename=None):
    liwc_fields_str = ', '.join(liwc_fields)
    liwc_valuetok_str = ', '.join(['?'] * len(liwc_fields))
    insert_query = "INSERT INTO %s(comment_id, %s) VALUES(?, %s)" \
                   % (liwctablename, liwc_fields_str, liwc_valuetok_str)
    return insert_query



def get_next_row(idsreader, datareader):
    comment_id = str(next(idsreader)[0])
    dat_row = next(datareader)
    data = list(map(float, dat_row[2:]))

    return comment_id, data



def liwc_to_db(dbdir, dbstem, idsfile, datafile, csvdir, liwctablename):
    datapath = get_path_(
        fdir=csvdir, fstem=datafile, fext='csv')
    idspath = get_path_(
        fdir=csvdir, fstem=idsfile, fext='ids')

    # print (datapath, idspath)
    conn = connect_to_db(dbdir=dbdir, dbstem=dbstem, dbpath=None)
    cursor = conn.cursor()

    size = 10000
    index = 0
    values = []
    with open(datapath, 'r') as datahandler, open(idspath, 'r') as idshandler:
        datareader = csv.reader(datahandler, delimiter=',')
        idsreader = csv.reader(idshandler)
        # we ignore the first row
        firstrow = next(datareader)
        
        # first two fields are filename and segment
        fields = firstrow[2:]

        create_liwc_table(conn, cursor, fields, liwctablename=liwctablename)
        insert_query = get_liwc_table_insert_query(fields, liwctablename=liwctablename)

        comment_id, data = get_next_row(idsreader, datareader)
        values.append([comment_id] + data)

        while True:
            try:
                comment_id, data = get_next_row(idsreader, datareader)
            except StopIteration:
                break
            values.append([comment_id] + data)

            if ((index + 1) % size) == 0:
                # in batches of <size> we write to liwc table
                cursor.executemany(insert_query, values)
                values = []
                conn.commit()
                #print('*')
                sys.stdout.flush()
            index += 1

    ## finally insert remaining values into table
    try:
        cursor.executemany(insert_query, values)
    except sqlite3.IntegrityError:
        raise
    conn.commit()
    


In [51]:
IDSFILE = 'minimal_seg0'
DATAFILE = 'LIWC_comment_test'
liwc_to_db(dbdir=DBDIR, dbstem=DBSTEM, idsfile=IDSFILE, datafile=DATAFILE, csvdir=DBDIR+'/'+SUBDIR,
                       liwctablename='liwc_comments')

Connecting to db with dbdir = ./preprocessing_test/sqlit_database
Connecting to db with dbstem = comment_test



## Sampling LIWC vectors to csv file

This is an optional step but is recommended so that the resulting experiments can be verified by rerunning.

Here we describe how to sample LIWC vectors from the database. There is the option to sample subsets of LIWC features, but the recommended approach is to sample the total vector and then to read from this file only those vectors you are interested in for your model. There is an option only to sample messages longer than a certain word-count.

In [52]:
QUERY_cm = 'SELECT comment_id, author_id FROM comments'
QUERY_liwc = 'SELECT * FROM liwc_comments'
QUERY_all_comments = 'SELECT %s FROM (SELECT * from liwc_comments as L JOIN (select comment_id,' \
                     'author_id from comments) as P on L.comment_id = P.comment_id)'

def get_results_from_db(dbdir, dbstem, features=None):
    
    conn = connect_to_db(dbdir=dbdir, dbstem=dbstem, dbpath=None)
    cursor = conn.cursor()

    if features is None:
        cursor.execute(QUERY_liwc)
        features_liwc = list(map(lambda x: x[0], cursor.description)) 
        features_all = ', '.join(list(map(lambda x: str(x), ['author_id']+features_liwc)))
        features = features_all
    else:
        features = ', '.join(list(map(lambda x: str(x), ['author_id', 'comment_id']+features)))
        
    cursor.execute(QUERY_all_comments % features)


    #cursor.execute(query)
    features = list(map(lambda x: x[0], cursor.description))


    liwc_results = [
        (int(features[0]), str(features[1]), list(map(lambda x: float(x), features[2:]))) for features in cursor.fetchall()]
    user_ids, comment_ids, liwc_features = zip(*liwc_results)

    df_liwc = pd.concat([pd.DataFrame(list(user_ids), columns=['user_id']),
                         pd.DataFrame(list(comment_ids), columns=['msg_id']),
                         pd.DataFrame(list(liwc_features), columns=features[2:])], axis=1)
    return df_liwc


In [53]:
output_file = 'output_test'

test_df = get_results_from_db(dbdir=DBDIR, dbstem=DBSTEM)
test_df.to_csv(DBDIR+'/'+SUBDIR+'/'+output_file+'.csv', index=False)

Connecting to db with dbdir = ./preprocessing_test/sqlit_database
Connecting to db with dbstem = comment_test


In [54]:
test_df

,user_id,msg_id,WC,WPS,Sixltr,Dic,funct,pronoun,ppron,i,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,1,1,15.0,15.0,26.67,80.00,53.33,6.67,6.67,0.0,...,13.33,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,1,2,29.0,14.5,6.90,86.21,65.52,10.34,10.34,0.0,...,3.45,0.0,0.00,0.0,0.0,0.0,0.0,3.45,0.0,0.0
2,1,3,25.0,25.0,24.00,84.00,64.00,20.00,20.00,12.0,...,12.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,2,4,30.0,15.0,16.67,83.33,60.00,10.00,0.00,0.0,...,6.67,0.0,3.33,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,2,5,26.0,13.0,15.38,69.23,53.85,0.00,0.00,0.0,...,3.85,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
5,2,6,33.0,16.5,18.18,87.88,57.58,15.15,9.09,0.0,...,6.06,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
6,2,7,25.0,12.5,20.00,88.00,60.00,20.00,16.00,0.0,...,12.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
